# Computational Carpentry - Project
#### Group K - Julian Barth, Marin Desgeans, Lucas Eichenberger
26 september 2025


#### Part A – Data structures and functions

The goal of this part is to create a function capable of calculating the molecular mass of any molecule. For this we will first load a periodic table containing all the elements' masses.

In [ ]:
import pandas as pd
import re
from collections import defaultdict

# 1. Load periodic_table.csv
df = pd.read_csv("periodic_table.csv")

We are now able to create a dictionnary where the keys are elements symbol and the values are atomic 
masses.

In [ ]:
# 2. Create dictionary
atomic_masses = dict(zip(df["Symbol"], df["AtomicMass"]))